In [1]:
import numpy as np
import cv2
from PIL import Image
from time import time

In [2]:
def processImg(img):
    img_tensor = np.array(img).astype(np.float32)                # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)              # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    # img_tensor /= 255.                                           # expects values in the range [0, 1]

    return img_tensor

In [3]:
def sigmoid(x):
    return np.exp(-np.logaddexp(0, -x))

In [4]:
import tensorflow as tf
interpreter = tf.lite.Interpreter(model_path="C:\\Users\\varun\\Downloads\\mobilenetv2_BSD.tflite")
interpreter.allocate_tensors()

In [5]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [6]:
cam = cv2.VideoCapture(0)  # Using a single camera (index 0)

if not cam.isOpened():
    print("Couldn't open camera")
    exit()

font = cv2.FONT_HERSHEY_SIMPLEX

In [7]:
def getPredictionFromRetAndFrame(ret, frame):
    if not ret:
        print("Can't receive frame from camera, skipping...")
        return  # Return instead of exit to continue with the loop
    
    timer = time()

    # Draw box first
    LINE_THICKNESS = 2  # Adjust thickness for smaller resolution
    LINE_COLOUR = (0, 0, 255)
    
    # Draw prediction box lines on the frame (adjusted for 640x480)
    cv2.line(frame, (0, 300), (640, 200), LINE_COLOUR, thickness=LINE_THICKNESS)
    cv2.line(frame, (640, 200), (580, 400), LINE_COLOUR, thickness=LINE_THICKNESS)
    cv2.line(frame, (580, 400), (0, 400), LINE_COLOUR, thickness=LINE_THICKNESS)
    
    # Crop and resize the frame
    if frame.shape[0] >= 480 and frame.shape[1] >= 640:  # Ensure frame is large enough (640x480)
        cropped = frame[120:480, 0:640]  # Crop the frame to focus on the area of interest
        resized = cv2.resize(cropped, (160, 160))  # Resize the cropped area to 160x160 for the model

        # Preprocess image and predict
        interpreter.set_tensor(input_details[0]['index'], processImg(resized))
        interpreter.invoke()

        # Convert raw to confidence level using sigmoid
        pred_raw = interpreter.get_tensor(output_details[0]['index'])
        pred_sig = sigmoid(pred_raw)
        pred = np.where(pred_sig < 0.5, 0, 1)
        timer = time() - timer

        # Display the prediction result
        readable_val = "Blind Spot" if pred[0][0] == 0 else "Clear"
        print(f"{readable_val} - Prediction took {round(timer, 3)}s")
        print("----------------------\n")

    else:
        print("Frame size too small for processing.")


In [8]:
while True:
    ret, frame = cam.read()
    getPredictionFromRetAndFrame(ret, frame)
    # Quit on 'q'
    cv2.imshow('Camera', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Release the camera and close windows
cam.release()


Blind Spot - Prediction took 0.0s
----------------------

Blind Spot - Prediction took 0.004s
----------------------

Blind Spot - Prediction took 0.0s
----------------------

Blind Spot - Prediction took 0.001s
----------------------

Blind Spot - Prediction took 0.005s
----------------------

Blind Spot - Prediction took 0.002s
----------------------

Blind Spot - Prediction took 0.004s
----------------------

Blind Spot - Prediction took 0.0s
----------------------

Blind Spot - Prediction took 0.0s
----------------------

Blind Spot - Prediction took 0.004s
----------------------

Blind Spot - Prediction took 0.0s
----------------------

Blind Spot - Prediction took 0.0s
----------------------

Blind Spot - Prediction took 0.003s
----------------------

Blind Spot - Prediction took 0.0s
----------------------

Blind Spot - Prediction took 0.0s
----------------------

Blind Spot - Prediction took 0.002s
----------------------

Blind Spot - Prediction took 0.001s
--------------------

: 